In [ ]:
import telegram
import pandas as pd
from selenium import webdriver
import numpy as np
import schedule
import datetime
import os
from time import sleep

In [ ]:
bot_token = '1886208392:AAE0RvGi62y1jhie6zadUou1vINiFpDMK34'
group_id = '@chartink_group'
bot = telegram.Bot(token=bot_token)

In [ ]:
url = 'https://chartink.com/screener/high-volume-stocks'

In [ ]:
def get_screened_stocks(url):

    driver = webdriver.Chrome(executable_path='/Users/entirety/Desktop/chromedriver')
    driver.get(url)
    table = pd.read_html(driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]').get_attribute('outerHTML'))
    driver.close()
    return table[0]

In [ ]:
def job():
    
    if os.path.isfile(f'screened_stocks_{datetime.date.today()}.csv'):

        earlier_screened_stocks = pd.read_csv(f'screened_stocks_{datetime.date.today()}.csv')
        earlier_screened_stocks_list = list(earlier_screened_stocks['Symbol'])

        #again scan for screened stocks at sometime if the file .csv of todays date already exists
        screened_stocks = get_screened_stocks(url)
        screened_stocks_list = list(screened_stocks['Symbol'])

        new_screened_stocks_list = []
        for stock in screened_stocks_list:
            if stock not in earlier_screened_stocks_list:
                new_screened_stocks_list.append(stock)

        for stock in new_screened_stocks_list:
                bot.send_message(chat_id=group_id,text=f'{stock} :: Screened at {datetime.datetime.now()}')

        screened_stocks = pd.concat([earlier_screened_stocks,screened_stocks],axis=0)
        screened_stocks = screened_stocks.drop_duplicates(subset=['Symbol'])
        screened_stocks.to_csv(f'screened_stocks_{datetime.date.today()}.csv',index=False)

    else:
        screened_stocks = get_screened_stocks(url)
        screened_stocks_list = list(screened_stocks['Symbol'])
        screened_stocks.to_csv(f'screened_stocks_{datetime.date.today()}.csv',index=False)

        for stock in screened_stocks_list:
            bot.send_message(chat_id=group_id,text=f'{stock} :: Screened at {datetime.datetime.now()}')

In [ ]:
while True:
    
    if (datetime.date.today().weekday() != 6) & (datetime.date.today().weekday() != 5):

        hours = [9,10,11,12,13,14,15]
        minutes = [0,15,30,45]

        for hour in hours:
            for minute in minutes:
                schedule_time = datetime.time(hour,minute)
                if (schedule_time >= datetime.time(9,15)) & (schedule_time <= datetime.time(15,15)):
                    schedule.every().day.at(str(schedule_time)).do(job)

        while True:
            schedule.run_pending()
            sleep(300)

    else:

        now_datetime = datetime.datetime.now()
        next_datetime = datetime.datetime.combine(datetime.date(now_datetime.year,now_datetime.month,now_datetime.day + 1),datetime.time(9,15))
        sleep((next_datetime - now_datetime).seconds)

@yashrj25 :: Twitter, Telegram, Instagram